# Spark Cluster Test
Testing parallel processing with multiple workers

In [ ]:
from pyspark.sql import SparkSession
import time

spark = SparkSession.builder \
    .appName("NotebookTest") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print(f"Spark {spark.version}")
print(f"Master: {spark.sparkContext.master}")

In [ ]:
text_df = spark.read.text("/opt/spark/scripts/test_wordcount.py")
print(f"Lines: {text_df.count()}")
text_df.show(5, truncate=False)

In [ ]:
data = [("Sahar", 24, "Data Engineer"),
        ("Kshitij", 27, "DevOps Engineer"),
        ("Rabina", 29, "ML Engineer"),
        ("Manu", 26, "Backend Developer"),
        ("Ola", 29, "Frontend Developer")]

df = spark.createDataFrame(data, ["name", "age", "role"])
df.show()

In [ ]:
df.groupBy("role").count().show()

In [ ]:
large_range = spark.range(0, 1000000)
result = large_range.filter(large_range.id % 2 == 0).count()
print(f"Even numbers: {result}")

In [ ]:
rdd = spark.sparkContext.parallelize(range(100))
squared = rdd.map(lambda x: x * x).collect()
print(f"Sum of squares (0-99): {sum(squared)}")

In [ ]:
spark.stop()